# Spark DataFrames

### Introduction

Now normally, when working with Pyspark, we work on a higher level of abstraction, which is to work with a dataframe.  In this lesson, we'll get started working with a dataframe.

### Introducing a Dataframe

The first step to creating a dataframe, is to initialize a spark session.

In [2]:
# You may need to reinstall pyspark in colab

!pip install fsspec --quiet
!pip install s3fs --quiet
!pip install pyspark --quiet

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

23/09/14 12:36:10 WARN Utils: Your hostname, Jeffreys-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.117.97.200 instead (on interface en0)
23/09/14 12:36:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/14 12:36:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Now, a spark session is very similar to a spark context, which we worked with in previous lessons.  A spark session, wraps around a spark context -- creating a new spark context if one doesn't currently exist. 

> So if we want to get the spark context from our session we can do so with the following.

In [4]:
spark.sparkContext

<SparkContext master=local[*] appName=Python Spark SQL basic example>

So this spark session is really just a thin wrapper around our spark context, which is one way for us to connect to our spark cluster.  The spark session is the other way.

Now let's use the spark session to create a dataframe.

### Creating a DataFrame

To create our dataframe, we can start with a list of dictionaries in Python.

In [5]:
movies = [{'index': 1,
  'title': 'Shazam!',
  'release_date': 1553299200,
  'genre': 'Comedy'}, {'index': 2,
  'title': 'Captain Marvel',
  'release_date': 1551830400,
  'genre': 'Action'},  {'index': 3,
  'title': 'Escape Room',
  'release_date': 1546473600,
  'genre': 'Horror'}, {'index': 4,
  'title': 'How to Train A Dragon',
  'release_date': 1546473600,
  'genre': 'Animation'}]

> So here we have a list of movies displaying the `title`, `release_date` and `genre` of each movie.

And then we can use the `createDataFrame` method on the spark session to create our dataframe.

In [6]:
movies_df = spark.createDataFrame(movies)

And we can view that dataframe by running the `show` method.

In [7]:
movies_df.show()

+---------+-----+------------+--------------------+
|    genre|index|release_date|               title|
+---------+-----+------------+--------------------+
|   Comedy|    1|  1553299200|             Shazam!|
|   Action|    2|  1551830400|      Captain Marvel|
|   Horror|    3|  1546473600|         Escape Room|
|Animation|    4|  1546473600|How to Train A Dr...|
+---------+-----+------------+--------------------+



So we can see from the above, that our dataframe organizes our data in a table.  It has associated our records with various columns.

We can also see the *schema on read* characteristic from spark.  That even without specifying a datatype, Spark was able to determine the datatype for each column.

In [8]:
movies_df.printSchema()

root
 |-- genre: string (nullable = true)
 |-- index: long (nullable = true)
 |-- release_date: long (nullable = true)
 |-- title: string (nullable = true)



### From DataFrame to RDD

Now a dataframe in Pyspark creates an RDD under the hood.

In [9]:
movies_df.rdd

MapPartitionsRDD[10] at javaToPython at NativeMethodAccessorImpl.java:0

In [46]:
movies_df.rdd.collect()

[Row(genre='Comedy', release_date=1553299200, title='Shazam!'),
 Row(genre='Action', release_date=1551830400, title='Captain Marvel'),
 Row(genre='Horror', release_date=1546473600, title='Escape Room'),
 Row(genre='Animation', release_date=1546473600, title='How to Train A Dragon')]

1. It's distributed

And that even though this looks like a unified dataset, it's really distributed across different nodes.

In [10]:
movies_df.rdd.getNumPartitions()

8

2. It's lazy

Because our dataset is built on RDDs, is also operates in lazy manner.  So for example, if we want to select all of the titles of an RDD, we'll use a `map` function to select the title from each row.  But because `map` is a transformation, it will not operate on our data, until we follow up with an action.

In [11]:
movies_df.rdd.map(lambda movie: movie['title'])

PythonRDD[11] at RDD at PythonRDD.scala:53

In [58]:
movies_df.rdd.map(lambda movie: movie['title']).collect()

['Shazam!', 'Captain Marvel', 'Escape Room', 'How to Train A Dragon']

If we perform the equivalent operation with a dataframe, the operation is also treated as a transformation.  Let's see this.  Below, we'll select the `title` of each record.

In [59]:
movies_df.select('title')

DataFrame[title: string]

So again, spark will not search through each of the records until an action is called.

In [60]:
movies_df.select('title').show()

+--------------------+
|               title|
+--------------------+
|             Shazam!|
|      Captain Marvel|
|         Escape Room|
|How to Train A Dr...|
+--------------------+



> So we can see that `show` is similar to `collect`. 

Let's do this one more time, this time with two columns.

In [61]:
movies_df.select(['title', 'genre']).show()

+--------------------+---------+
|               title|    genre|
+--------------------+---------+
|             Shazam!|   Comedy|
|      Captain Marvel|   Action|
|         Escape Room|   Horror|
|How to Train A Dr...|Animation|
+--------------------+---------+



> So to select multiple columns, we pass through a list of columns.

4. Only Coarse Grained Operations

Remember that with our RDDs, we only have coarse grained methods available to us -- those methods like `map` or `filter` that operate across a dataset.  This makes things a little tricky if we want to just select a single row.  For example, we may think that with our dataframe above, we want to select the entry at a specific index.  With our dataframe, the only way to do this is to use something akin to the filter method -- where we ask to *select* the rows that have an id of 1.  But we'll learn how to do that with our dataframe in a future lesson.

### Summary

In this lesson we learned how to create a DataFrame in Spark.  We do so, not through our Spark context but by creating a Spark session.  

In [102]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [103]:
movies_df = spark.createDataFrame(movies)

We then saw that our dataframe is really just a simpler interface for interacting with our resilient distributed dataset.  And because of this, it contains the same features of our RDD: it's distributed, it's lazy, and allows for only coarse grained transformations.

This knowledge gave us some insight into how to interact with our dataframe.  So we saw that to select specific columns, we have to use the `select` method, which operates as a `transformation` and then use the `show` method as our action. 

In [104]:
movies_df.select(['title', 'genre']).show()

+--------------------+---------+
|               title|    genre|
+--------------------+---------+
|             Shazam!|   Comedy|
|      Captain Marvel|   Action|
|         Escape Room|   Horror|
|How to Train A Dr...|Animation|
+--------------------+---------+



### Resources

[Pyspark operations](https://hendra-herviawan.github.io/)

[Pyspark DataFrame Rows and Columns](https://hendra-herviawan.github.io/pyspark-dataframe-row-columns.html)

[Creating a Dataframe](https://neapowers.com/pyspark/createdataframe-todf/)

[Spark by Examples](https://sparkbyexamples.com/pyspark-tutorial/#pyspark-dataframe)

[Data Partitioning Spark](https://kontext.tech/column/spark/296/data-partitioning-in-spark-pyspark-in-depth-walkthrough)


[DataBricks RDD to Dataframe](https://databricks.com/glossary/what-is-rdd)

[DataFrame Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)